<img src="./ESSENCE.png" width=700px>

### Reflection pattern

We generate a response with our first query and then add this content to the request for a critique or further refinement.

The first request may be a standard request as in this case to generate some Python Code and the system prompt reflects this.

Then with the returned content, we create a second request with the content included, (RAG), with some instruction to critique it or refine it.

This can be repeated until we reach certain criteria or MAX_ITERATIONS, whichever comes first.

Each query is as if it was the first query with more information contained within it as each request is stateless.


In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
client = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:10]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-TV


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [4]:
content = """
You are a Python programmer tasked with generating high quality Python code. Your task is to Generate the best content possible for the user's request. If the user requests critique, respond with a revised version of your previous attempt.
"""

generation_chat_history = [{"role": "system", "content": content}]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [6]:
user_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [7]:
display_markdown(user_code, raw=True)

Certainly! Below is a Python implementation that demonstrates how to request data from an API using the `requests` library. The example shows how to make a GET request, handle the response, and check for errors.

```python
import requests

def fetch_data_from_api(url, params=None):
    """
    Fetch data from a given API URL.
    
    :param url: The API endpoint to request data from.
    :param params: Optional query parameters to include in the request.
    :return: JSON response as a dictionary if successful, or None if failed.
    """
    try:
        response = requests.get(url, params=params)
        
        # Check if the request was successful
        response.raise_for_status()  # Raises an HTTPError for bad responses (4xx and 5xx)
        
        # Parse JSON response
        return response.json()
        
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None

if __name__ == "__main__":
    # Example usage
    api_url = "https://jsonplaceholder.typicode.com/posts"
    # Optional parameters, e.g., to filter results
    params = {'userId': 1}
    
    data = fetch_data_from_api(api_url, params)
    
    if data:
        print("Fetched Data:")
        for post in data:
            print(f"ID: {post['id']}, Title: {post['title']}")
    else:
        print("Failed to fetch data from the API.")
```

### Explanation:
1. **Function Definition**: The `fetch_data_from_api` function takes two arguments: the `url` of the API endpoint and optional `params` for query parameters.
2. **Making the Request**: The `requests.get()` method is used to make the GET request to the specified URL. If parameters are provided, they are added to the query string.
3. **Error Handling**: The `raise_for_status()` method checks if the request was successful (status codes 200-299). If the response indicates an error, it raises an exception.
4. **Response Handling**: The response is parsed as JSON and returned as a dictionary. If an exception occurs, an error message is printed, and `None` is returned.
5. **Example Usage**: In the `if __name__ == "__main__":` block, the function is called with an example API URL and parameters. The resulting data is printed if the fetch is successful.

Make sure to install the `requests` library if you haven't already by running:
```bash
pip install requests
```

Feel free to modify the `api_url` and `params` to suit your specific API needs!

## Reflection Step


This is equivalent to asking a follow up question in say ChatGPT


In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.


In [10]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [11]:
# Display

display_markdown(critique, raw=True)

Your implementation for fetching data from an API using the `requests` library is well-structured and generally follows good practices. Here are some critiques and recommendations for improving the code:

### Positive Aspects:
1. **Clear Documentation**: The docstring for the `fetch_data_from_api` function is informative, providing a clear description of parameters and return values.
2. **Error Handling**: Using `raise_for_status()` effectively catches HTTP errors and allows for graceful error handling.
3. **Usage Example**: Including an example usage of the function in the `__main__` block makes your code easy to understand and test.

### Areas for Improvement:
1. **Granular Error Handling**: You currently catch all `requests.RequestException` errors. Consider refining error handling to provide more specific feedback based on the type of error (e.g., connection issues, timeout, HTTP errors). This could be done by catching specific exceptions like `HTTPError`, `Timeout`, or `ConnectionError`.
   
    ```python
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # Bad response
    except requests.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")  # Network issue
    except requests.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")  # Request timed out
    except Exception as err:
        print(f"An unexpected error occurred: {err}")  # Other issues
    ```

2. **Logging Instead of Printing**: For better maintainability, consider using the `logging` library instead of printing errors directly. This way, you can control the logging levels and formats and redirect logs to files if necessary.
   
    ```python
    import logging
    
    logging.basicConfig(level=logging.INFO)  # Configure logging level
    ...
    except requests.RequestException as e:
        logging.error(f"An error occurred: {e}")
        return None
    ```

3. **Timeouts**: In API requests, it's good practice to specify a timeout. Without it, your code might hang indefinitely if the server doesn't respond. You can add a `timeout` parameter to the `requests.get()` call:

    ```python
    response = requests.get(url, params=params, timeout=10)  # Timeout set to 10 seconds
    ```

4. **Type Hinting**: Use type hints for function parameters and return types for improved readability and better integration with IDEs that support type checking.

    ```python
    from typing import Optional, Dict, Any

    def fetch_data_from_api(url: str, params: Optional[Dict[str, Any]] = None) -> Optional[Dict[str, Any]]:
    ```

5. **Storing API Key and Sensitive Information**: If you plan to use APIs that require authentication or API keys, consider using environment variables or configuration files to manage sensitive information securely.

### Example Revised Function:
Here’s a revised version of your `fetch_data_from_api` function incorporating the recommendations:

```python
import requests
import logging
from typing import Optional, Dict, Any

logging.basicConfig(level=logging.INFO)

def fetch_data_from_api(url: str, params: Optional[Dict[str, Any]] = None) -> Optional[Dict[str, Any]]:
    try:
        response = requests.get(url, params=params, timeout=10)  # Using a timeout
        response.raise_for_status()
        return response.json()
    except requests.HTTPError as http_err:
        logging.error(f"HTTP error occurred: {http_err}")
    except requests.ConnectionError as conn_err:
        logging.error(f"Connection error occurred: {conn_err}")
    except requests.Timeout as timeout_err:
        logging.error(f"Timeout error occurred: {timeout_err}")
    except Exception as err:
        logging.error(f"An unexpected error occurred: {err}")
    return None
```

These modifications will enhance the reliability, maintainability, and usability of your API data fetching function. Nice work overall!

Add CRITIQUE to chat...


In [12]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [13]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [14]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)

Thank you for the constructive feedback! Here's a revised version of the initial implementation, incorporating your recommendations for better error handling, logging, specifying timeouts, and type hinting:

```python
import requests
import logging
from typing import Optional, Dict, Any

# Set up logging configuration
logging.basicConfig(level=logging.INFO)

def fetch_data_from_api(url: str, params: Optional[Dict[str, Any]] = None) -> Optional[Dict[str, Any]]:
    """
    Fetch data from a given API URL.
    
    :param url: The API endpoint to request data from.
    :param params: Optional query parameters to include in the request.
    :return: JSON response as a dictionary if successful, or None if failed.
    """
    try:
        # Set a timeout for the request
        response = requests.get(url, params=params, timeout=10)
        # Check if the request was successful
        response.raise_for_status()
        # Parse JSON response
        return response.json()

    except requests.HTTPError as http_err:
        logging.error(f"HTTP error occurred: {http_err}")  # Bad response
    except requests.ConnectionError as conn_err:
        logging.error(f"Connection error occurred: {conn_err}")  # Network issue
    except requests.Timeout as timeout_err:
        logging.error(f"Timeout error occurred: {timeout_err}")  # Request timed out
    except Exception as err:
        logging.error(f"An unexpected error occurred: {err}")  # Other issues

    return None

if __name__ == "__main__":
    # Example usage
    api_url = "https://jsonplaceholder.typicode.com/posts"
    params = {'userId': 1}
    
    data = fetch_data_from_api(api_url, params)
    
    if data:
        logging.info("Fetched Data:")
        for post in data:
            logging.info(f"ID: {post['id']}, Title: {post['title']}")
    else:
        logging.error("Failed to fetch data from the API.")
```

### Key Changes:
1. **Granular Error Handling**: Each type of error is caught separately, providing more specific logging.
2. **Logging**: Used the `logging` library for better maintainability and control over output, rather than using print statements.
3. **Timeout**: A timeout of 10 seconds is set for the API request to avoid indefinite hanging.
4. **Type Hinting**: Added type hints to improve clarity and IDE support.

This revised version follows best practices and is more robust for handling API requests. Thank you for your insightful critiques!

<br>

## We can of course make this a Class...
